# Transitioning to Franklin from Raster Foundry

This notebook contains a walkthrough for interacting with Franklin, a [STAC API server](https://github.com/radiantearth/stac-api-spec) implementation, over HTTP.

The purpose of Franklin is to replace the data management and visualization capabilities that you currently rely on Raster Foundry for.

While with Raster Foundry you created _projects_ and used cloud-optimized GeoTIFFs to create _scenes_, with Franklin you'll
create [_collections_](../create_collection.py) and use cloud-optimized GeoTIFFs to create [_items_](../create_item.py).

## Creating a collection

In Raster Foundry, a `project` was the entity responsible for associating several COGs so you could view a tile layer that they all contribute to. In Franklin, `collections` serve the same purpose. You can read more about collections in the [collection specification](https://github.com/radiantearth/stac-spec/blob/v1.0.0/collection-spec/collection-spec.md).

To create a collection, you can run the `create_collection.py` script:

```bash
$ python create_collection.py --help
Usage: create_collection.py [OPTIONS] NAME

Options:
  -d, --description TEXT          Collection description. This should be a
                                  short punchy phrase that describes what
                                  kinds of items can be found in this
                                  collection
  --bbox BBOX                     Bounding box in comma-separated lower left
                                  x, lower left y, upper right x, upper right
                                  y format. This will expand as you add items,
                                  so picking a small bbox in the appropriate
                                  area makes sense to start.  [required]
  --start-date [%Y-%m-%d|%Y-%m-%dT%H:%M:%S|%Y-%m-%d %H:%M:%S]
                                  Datetime marking the beginning of coverage
                                  for this collection. Provide as an ISO 8601
                                  timestamp without a time zone -- it will be
                                  parsed as UTC. Defaults to now.  [required]
  --end-date [%Y-%m-%d|%Y-%m-%dT%H:%M:%S|%Y-%m-%d %H:%M:%S]
                                  Datetime marking the end of coverage for
                                  this collection. Provide as an ISO 8601
                                  timestamp without a time zone -- it will be
                                  parsed as UTC. Defaults to now.  [required]
  --license TEXT                  License of the underlying data. You can use
                                  identifiers from
                                  https://spdx.github.io/license-list-data/ in
                                  this field or the string "proprietary"
  --api-host TEXT                 The root of the STAC API you'd like to
                                  interact with. Defaults to localhost:9090
  --api-scheme TEXT               The scheme to use for API communication.
                                  Defaults to http
  --help                          Show this message and exit.
```

Creating a collection requires you to provide:

- `--start-date` and `--end-date` arguments. These arguments are formatted like ISO 8601 dates and define the [_temporal extent_](https://github.com/radiantearth/stac-spec/blob/v1.0.0/collection-spec/collection-spec.md#temporal-extent-object) of the collection. To start, you can specify a relatively narrow temporal extent -- as you add items it will automatically expand.
- a `--bbox` argument. This argument defines the [_spatial extent_](https://github.com/radiantearth/stac-spec/blob/v1.0.0/collection-spec/collection-spec.md#spatial-extent-object) of the collection. Similar to the temporal extent, it's sufficient to define a narrow area that you know you'll cover, since this will also expand as you add items to the collection.

Let's create a collection called `demo-collection` now. This collection will have a temporal extent covering July 2021 and a bbox covering a small point in the Canary Islands.

In [ ]:
%%bash
python ../create_collection.py \
    demo-collection \
    --start-date 2021-07-01 \
    --end-date 2021-08-01 \
    --bbox -15.63,27.98,-15.60,27.99

After creating the collection, we can inspect it at `localhost:9090/collections/demo-collection`:

In [ ]:
import requests

requests.get("http://localhost:9090/collections/demo-collection").json()

## Creating items

In Raster Foundry, you created _scenes_ with pointers to where the server could find a cloud-optimized GeoTIFF. In Franklin, we'll instead create _items_ and store information about where to find the COG in the item's [_assets_](https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/item-spec.md#asset-object). You can read more about items in the [items specification](https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/item-spec.md).

You create items with the `create_item.py` script:

```bash
$ python create_item.py --help
Usage: create_item.py [OPTIONS] ITEM_ID [%Y-%m-%d|%Y-%m-%dT%H:%M:%S|%Y-%m-%d
                      %H:%M:%S] COG_PATH COLLECTION_ID

Options:
  --eo-data PATH       Path to JSON containing information to fill in the EO
                       extension for this item. You can read more about the EO
                       extension at https://github.com/stac-
                       extensions/eo/tree/v1.0.0
  --cloud-cover FLOAT  Estimated cloud cover of the image
  --api-host TEXT      The root of the STAC API you'd like to interact with.
                       Defaults to localhost:9090
  --api-scheme TEXT    The scheme to use for API communication. Defaults to
                       http
  --help               Show this message and exit.
```

The required options for an item are an id, a date time, a path to a COG that you can read, and a collection to hold the item. While you could create a scene outside of a project in Raster Foundry, in Franklin, **you cannot create an item outside of a collection**.

Let's create two items, both of which point to COGs of Sentinel-2 imagery over the Canary Islands. You'll need to set an `AWS_PROFILE` environment variable to read the COGs linked here, but it doesn't matter which one.

In [ ]:
%%bash
AWS_PROFILE=default \
python ../create_item.py \
    demo-item-1 \
    2021-07-18 \
    s3://rasterfoundry-production-data-us-east-1/demo-cogs/s2-canary-islands-rgb-cog.tif \
    demo-collection

In [ ]:
%%bash
AWS_PROFILE=default \
python ../create_item.py \
    demo-item-2 \
    2021-07-18 \
    s3://rasterfoundry-production-data-us-east-1/demo-cogs/s2-canary-islands-rgb-cog-2.tif \
    demo-collection

After creating the items, we can verify that the collection knows about them by querying its `items` endpoint:

In [ ]:
resp = requests.get("http://localhost:9090/collections/demo-collection/items").json()
resp

And if we're specifically interested in the `assets` each item has, we can extract that information from the response's `features` key:

In [ ]:
[x["assets"] for x in resp["features"]]

## Viewing tiles

Note that each of the items above has a key named `cog` in its `assets` property. Because items can have multiple assets, several of which can be tiled, Franklin requires one additional layer in order to produce tiles for a collection that wasn't required in Raster Foundry. This layer is a _mosaic definition_.

A _mosaic definition_ is a JSON document that describes which images you want to contribute to the resulting tiles and offers some metadata that could be useful for clients.

Here's an example document:

```json
{
    "id": "d9b4db0f-7d80-4d96-b176-15a1400c01de",
    "description": null,
    "center": [
        -16.16291,
        28.2916,
        14
    ],
    "items": [
        {
            "itemId": "demo-item-1",
            "assetName": "cog"
        },
        {
            "itemId": "demo-item-2",
            "assetName": "cog"
        }
    ],
    "minZoom": 2,
    "maxZoom": 30,
    "bounds": [
        -180,
        -90,
        180,
        90
    ]
}
```

You can create a mosaic definition with the `create_mosaic_definition.py` script:

```bash
$ python create_mosaic_definition.py --help
Usage: create_mosaic_definition.py [OPTIONS]

Options:
  --collection-id TEXT    Which collection holds the items of this mosaic
                          [required]
  --items-csv PATH        Path to a CSV of (itemId, assetName) pairs
                          [required]
  -d, --description TEXT  Description of the what this mosaic represents
  --center CENTER         x,y (longitude,latitude) of the center of this
                          mosaic  [required]
  --center-zoom TEXT      The zoom a client should choose on first load of
                          this mosaic  [required]
  --min-zoom INTEGER      Minimum zoom level clients should view to consume
                          this mosaic
  --max-zoom INTEGER      Maximum zoom level clients should view to consume
                          this mosaic
  --bounds BBOX           Bounding box in which this mosaic has coverage
  --api-host TEXT         The root of the STAC API you'd like to interact
                          with. Defaults to localhost:9090
  --api-scheme TEXT       The scheme to use for API communication. Defaults to
                          http
  --help                  Show this message and exit.
```

Creating a mosaic definition requires you to provide:

- a `collection-id`. This argument indicates which collection the items you provide are associated with. For us this will be `demo-collection`.
- an `--items-csv` path. This argument says where to find a CSV where each row is an item ID in the collection and an asset name that exists on that item. We'll create the items CSV below in a moment.
- a `--center`. This argument is a point that clients can consume to choose where to center maps while viewing this collection.
- a `--center-zoom`. This argument tells clients what zoom level they should choose when they begin to view this mosaic.

Note that you don't need to provide an `id`, since we can create one on the fly when we POST the JSON document.

To create the items CSV, we'll write our demo items' IDs and assets into a file:

In [ ]:
import csv

item_assets = [("demo-item-1", "cog"), ("demo-item-2", "cog")]
with open("../item-assets.csv", "w", newline="") as outf:
    writer = csv.writer(outf)
    for ia in item_assets:
        writer.writerow(ia)

We already know everything else we need, so we can create our mosaic definition:

In [ ]:
%%bash
python ../create_mosaic_definition.py \
    --collection-id demo-collection \
    --items-csv ../item-assets.csv \
    --center -15.8,28.4 \
    --center-zoom 12 \

We can view the JSON representation of your mosaic under its associated collection. For example, we can copy the `/collections/demo-collection/mosaic/<id>` URL segment above to view the mosaic you just created:

In [ ]:
segment = "PASTE YOUR SEGMENT HERE"
requests.get(f"http://localhost:9090{segment}").json()

Or you can view all of the mosaics you've created for a collection by sending a GET request to its `/mosaic` route:

In [ ]:
requests.get(f"http://localhost:9090/collections/demo-collection/mosaic").json()

We can use the mosaic JSON information in clients to tell libraries like Leaflet about the min and max zoom for your layer, where to center the map, and in what bounding box data is defined.

If you actually want to view tiles, you can put the tile URL above (the link ending in `/{z}/{x}/{y}`) into anything that can view TMS raster tiles. Two good options are QGIS and geojson.io. To view tiles in QGIS, create a new connection under XYZ Tiles in the Browser pane. To view tiles in geojson.io, choose "Add map layer" under "Meta" and paste your URL. Note you may have to fix the text entry in geojson.io, since it has a habit of URL-encoding what you paste even where the template braces are appropriate.

You can control the visualization with `redBand`, `greenBand`, and `blueBand` parameters for RGB selection and with `upperQuantile` and `lowerQuantile` parameters for controlling the histogram stretch.